In [ ]:
from keras.applications import resnet50

Using TensorFlow backend.


In [ ]:
model = resnet50.ResNet50(weights='imagenet',include_top=False , input_shape=(224,224,3))

In [ ]:
model.input



In [ ]:
for layers in model.layers :
    layers.trainable = False 

In [ ]:
model.layers[6].trainable

In [ ]:
from keras.layers import Dense 
from keras.models import Model
from keras.layers import Flatten
from keras.models import Sequential

In [ ]:
top_model = model.output


In [ ]:

top_model= Flatten()(top_model)

top_model= Dense(units=1024,activation='relu')(top_model)


In [ ]:
top_model = Dense(units=512,activation='relu')(top_model)


In [ ]:
top_model= Dense(units=10,activation='softmax')(top_model)

In [ ]:
newmodel = Model(input=model.input , output=top_model )

In [ ]:
newmodel.summary()

In [ ]:
img_rows, img_cols = 224, 224 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Mewtwo/train/'
validation_data_dir = 'Mewtwo/test/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("monkey_breed_mobileNet.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1097
nb_validation_samples = 272

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)